# Optimise Query

In [ ]:
# Install requirements

!pip install -q llama-index==0.12.21 openai==1.59.8 tiktoken==0.8.0  huggingface-hub==0.30 chromadb==0.6.0 llama-index-vector-stores-chroma==0.4.1 llama-index-llms-gemini==0.4.1 llama-index-finetuning==0.3.2 llama-index-postprocessor-cohere-rerank==0.3.0 llama-index-embeddings-huggingface==0.5.4 llama-index-embeddings-cohere==0.5.0 cohere==5.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.1/481.1 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.4 MB/s e

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["CO_API_KEY"] = userdata.get('co_api_key')
cohere_key = userdata.get('co_api_key')

In [ ]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load model

In [ ]:
# prompt: download a finetuned model from hugging face, repor vicpada/finetuned_embed_model
from huggingface_hub import snapshot_download

snapshot_download(repo_id="vicpada/finetuned_embed_model_full", repo_type="model", local_dir="./finetuned_embed_model_full")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/593k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

'/content/finetuned_embed_model_full'

In [ ]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.adapter import AdapterEmbeddingModel

# Load the Base model without fine-tuning
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# Load the Fine-tuned model.
embed_model = AdapterEmbeddingModel(base_embed_model, "finetuned_embed_model_full")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = embed_model

# Download Vector DB

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
            repo_id="vicpada/AzureArchitectKnowledgeFull",
            local_dir="azure-architect",
            repo_type="dataset",
        )

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

header.bin:   0%|          | 0.00/100 [00:00<?, ?B/s]

length.bin:   0%|          | 0.00/780k [00:00<?, ?B/s]

link_lists.bin:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

chroma.sqlite3:   0%|          | 0.00/5.20G [00:00<?, ?B/s]

data_level0.bin:   0%|          | 0.00/327M [00:00<?, ?B/s]

index_metadata.pickle:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

'/content/azure-architect'

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex



# Load the vector store from the local storage.
chroma_path = './azure-architect'
db = chromadb.PersistentClient(path=chroma_path, settings=chromadb.config.Settings(allow_reset=True))
db.list_collections()
chroma_collection = db.get_collection("azure-architect")
chroma_collection.peek()
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

# Define the Cohere Reranking object to return only the first two highest ranking chunks.

cohere_rerank3 = CohereRerank(top_n=2, api_key=cohere_key,model = 'rerank-english-v3.0')

In [ ]:
# Define the ServiceCotext object to tie the LLM for generating final answer,
# and the embedding model to help with retrieving related nodes.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(
    similarity_top_k=400, node_postprocessors=[cohere_rerank3]
)

#res = query_engine.query("When to use Azure Functions instead of App Service?")
res = query_engine.query("list the most known patterns from azure architecture. Explain with details. Use only the context provided")
res.response

print(res.source_nodes)

[NodeWithScore(node=TextNode(id_='5513e98d-b3ff-40e5-9eb7-201f1e2869fe', embedding=None, metadata={'url': 'https://learn.microsoft.com/en-us/azure/architecture/networking/guide/spoke-to-spoke-networking', 'title': 'Patterns for inter-spoke networking - Azure Architecture Center | Microsoft Learn', 'tokens': 6417, 'retrieve_doc': True, 'source': 'azure-architecture'}, excluded_embed_metadata_keys=['url', 'tokens', 'retrieve_doc', 'source', 'context_added'], excluded_llm_metadata_keys=['title', 'tokens', 'retrieve_doc', 'source', 'context_added'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3325b42b-9e6f-54c7-8882-b5078237e102', node_type='4', metadata={'url': 'https://learn.microsoft.com/en-us/azure/architecture/networking/guide/spoke-to-spoke-networking', 'title': 'Patterns for inter-spoke networking - Azure Architecture Center | Microsoft Learn', 'tokens': 6417, 'retrieve_doc': True, 'source': 'azure-architecture'}, hash='b05298929d6f54c578805a0974e4584ef23

# Evaluate Cohere rerank

In [ ]:
from huggingface_hub import hf_hub_download
from llama_index.finetuning.embeddings.common import (
    EmbeddingQAFinetuneDataset,
)

# Download the evaluation dataset
hf_hub_download(repo_id="vicpada/AzureRAGEval", filename="rag_eval_dataset2.json", repo_type="dataset", local_dir=".")
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json("./rag_eval_dataset2.json")

rag_eval_dataset2.json:   0%|          | 0.00/27.8M [00:00<?, ?B/s]

In [ ]:
import pandas as pd


#  A simple function to show the evaluation result.
def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()
    precision = full_df["precision"].mean()
    recall = full_df["recall"].mean()
    ap = full_df["ap"].mean()
    ndcg = full_df["ndcg"].mean()

    metric_df = pd.DataFrame(
        {"Retriever": [name], "Hit Rate": [hit_rate], "MRR": [mrr], "Precision": [precision], "Recall":[recall],"AP":[ap],"NDCG":[ndcg]}
    )

    return metric_df

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]


# We can evaluate the retievers with different top_k values.
for i in [2,10,20,50,100,200,400]:
    rerank_retriever = index.as_retriever(
        similarity_top_k=i, node_postprocessors=[cohere_rerank3]
    )
    rerank_retriever_evaluator = RetrieverEvaluator.from_metric_names(
        metrics, retriever=rerank_retriever
    )
    rerank_eval_results = await rerank_retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Rerank top_{i}", rerank_eval_results))
    standard_retriever = index.as_retriever(
        similarity_top_k=i
    )
    standard_retriever_evaluator = RetrieverEvaluator.from_metric_names(
        metrics, retriever=standard_retriever
    )
    eval_results = await standard_retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Standard top_{i}", eval_results))

      Retriever  Hit Rate     MRR  Precision  Recall      AP      NDCG
0  Rerank top_2    0.0217  0.0184    0.01085  0.0217  0.0184  0.019264
        Retriever  Hit Rate     MRR  Precision  Recall      AP      NDCG
0  Standard top_2    0.0217  0.0184    0.01085  0.0217  0.0184  0.019264
       Retriever  Hit Rate       MRR  Precision  Recall        AP      NDCG
0  Rerank top_10    0.0443  0.023128   0.004438  0.0443  0.023128  0.028113
         Retriever  Hit Rate       MRR  Precision  Recall        AP      NDCG
0  Standard top_10    0.0443  0.023128   0.004438  0.0443  0.023128  0.028113
       Retriever  Hit Rate       MRR  Precision  Recall        AP     NDCG
0  Rerank top_20     0.068  0.027694   0.003405   0.068  0.027694  0.03654
         Retriever  Hit Rate       MRR  Precision  Recall        AP     NDCG
0  Standard top_20     0.068  0.027694   0.003405   0.068  0.027694  0.03654
       Retriever  Hit Rate       MRR  Precision  Recall        AP      NDCG
0  Rerank top_50    0.11